In [1]:
import os
import shutil
# 這兩個套件是為了要將訓練資料的路徑讀取工具也引進來

os.environ['TF_USE_LEGACY_KERAS'] = '1'

import tensorflow as tf # 就是 Tensorflow 不用多說
import tensorflow_hub as hub # Tensorflow bert 社群
import tensorflow_text as text # 前處理
from official.nlp import optimization  # 優化工具

import matplotlib.pyplot as plt # 畫圖用的套件

tf.get_logger().setLevel('ERROR') # Debug的工具，也可以不寫.


2025-01-14 18:37:03.699780: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.6/lib64:
2025-01-14 18:37:03.699827: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/ysh/miniconda3/envs/3.9/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/ysh/miniconda3/envs/3.9/l

In [2]:
from core.general import *

info(tf.__version__)
tf.test.is_gpu_available()

INFO  2.8.4


2025-01-14 18:37:08.721891: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-14 18:37:08.916194: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-01-14 18:37:08.920945: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.6/lib64:
2025-01-14 18:37:08.921172: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.s

False

In [3]:
# # 這個是資料集的下載連結，如果你複製這段 url 到瀏覽器上，也可以下載的到
# url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

# dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
#                                   untar=True, cache_dir='.',
#                                   cache_subdir='')

# # 所謂的 os.path.join 就是把兩個路徑結合起來的意思，如果路徑是 ./data/，join之後就會變成 ./data/aclImdb，而以下所做的，就是要整合資料路徑，方便之後模型讀取
# dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

# train_dir = os.path.join(dataset_dir, 'train')

# # remove unused folders to make it easier to load the data
# remove_dir = os.path.join(train_dir, 'unsup')
# shutil.rmtree(remove_dir)


In [4]:
from core.general import *
ls('../data')

import pandas as pd
f = pd.read_csv('../data/filtered_train_data.csv')
f = f[['Statement', 'tags']]
f['isDP'] = ['dp' in i for i in f['tags'].tolist()]
f = f.drop('tags', axis = 1)
f

,Statement,isDP
0,watch new seri squid game mashtali soroush dec...,False
1,today math lesson teacher told vovochka euler ...,False
2,sequenc integ call good max min sequenc integ ...,True
3,petya got interest grammar third year school i...,False
4,nezzar design brand new game hidden permut sha...,False
...,...,...
7971,given array integ find maximum possibl valu am...,True
7972,polycarp poor memori day rememb 3 differ lette...,False
7973,hard version problem differ easi version versi...,False
7974,two chess piec rook knight stand standard ches...,False


In [5]:
pre = f.to_dict()
# for i in train:
# debug(pwd())
cd(root)
cd('aclImdb/train')
for tag in ['pos', True], ['neg', False]:
    cd(tag[0])
    for i in range(len(f['Statement'])):
        if pre['isDP'][i] ^ tag[1]: continue;
        write_to_file(f'{i}.txt', pre['Statement'][i])
    cd('..')
cd('../..')

In [6]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

info('Loading Training Data.')
raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

info('Loading Validating Data.')
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

info('Loading Testing Data.')
test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


INFO  Loading Training Data.
Found 7976 files belonging to 2 classes.
Using 6381 files for training.
INFO  Loading Validating Data.


2025-01-14 18:37:10.508175: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-01-14 18:37:10.509649: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Found 7976 files belonging to 2 classes.
Using 1595 files for validation.
INFO  Loading Testing Data.
Found 1995 files belonging to 2 classes.


In [7]:
for text_batch, label_batch in train_ds.take(1):
    print(f'Review: {text_batch.numpy()[0]}')
    for i in range(3):
        label = label_batch.numpy()[i]
        print(f'Label : {label} ({class_names[label]})')

Review: b'interact n point cartesian plane denot point coordin integ inclus three point collinear say point vertic convex polygon word exist permut integ 1 n polygon convex vertic list give number n hide coordin point task guess permut ask multipl queri queri give khanh 4 integ j k either 1 2 j k three distinct indic 1 n inclus respons khanh tell 1 area triangl multipli 2 2 sign cross product two vector recal cross product vector vector b integ sign number 1 posit otherwis proven cross product obtain queri 0 ask 3 n note khanh fix coordin point chang answer queri need guess coordin permut equal 1 indic vertic list order'
Label : 0 (neg)
Label : 0 (neg)
Label : 0 (neg)


2025-01-14 18:37:11.029202: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [8]:
tfhub_handle_encoder = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1" # 這是預訓練模型
tfhub_handle_preprocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3" # 這是 BERT 前處理需要用的模型

In [9]:
debug(pre['Statement'][0])

DEBUG watch new seri squid game mashtali soroush decid hold squid game soroush agre host provid money winner prize mashtali becam front man player regist play game win great prize mashtali found huge winner prize go decid kill elimin player could take money evil mashtali go elimin player unroot tree n vertic everi player 2 special vertic one oper mashtali choos vertex v tree remain player find vertex w simpl path closest v player mashtali wonder minimum number oper perform remov everi player game take money sinc think money could solv problem ask help


In [10]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess) # 呼叫前處理模型
text_test = [pre['Statement'][0]] # 先輸入簡單的句子看看會變成什麼樣子
text_preprocessed = bert_preprocess_model(text_test)
print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_word_ids', 'input_mask', 'input_type_ids']
Shape      : (1, 128)
Word Ids   : [  101  3422  2047 14262  2072 26852  2208 16137 22893  3669  2061 13288]
Input Mask : [1 1 1 1 1 1 1 1 1 1 1 1]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [11]:
with tf.device('/CPU:0'):
    bert_model = hub.KerasLayer(tfhub_handle_encoder)
    debug('---')
    bert_results = bert_model(text_preprocessed)
print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')


DEBUG ---
Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.99807113  0.7797979  -0.0370684   0.3995302   0.07245599  0.40720418
  0.9682053  -0.99730515 -0.7465364  -0.88527596 -0.4546446   0.83189595]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[-9.28442657e-01  1.04626226e+00 -5.84454834e-01 ... -7.04503477e-01
  -2.72823095e-01 -4.49318707e-01]
 [-2.51816779e-01  4.96864468e-02  6.76903844e-01 ... -2.99022317e-01
  -6.40931189e-01  1.29349545e-01]
 [-5.00569940e-01 -1.29153579e-03 -7.94892311e-01 ... -1.20398566e-01
  -3.05489123e-01  2.88654476e-01]
 ...
 [-1.28959584e+00  5.35864770e-01 -3.78003418e-02 ...  1.31041276e+00
   7.49447703e-01  1.61270738e-01]
 [-7.77777791e-01 -9.57850695e-01 -1.97193012e-01 ... -5.76900840e-01
   3.61313820e-01  6.66653514e-01]
 [-5.03935218e-01  1.62717581e-01 -1.52471632e-01 ...  1.60439789e-01
  -7.64720619e-01 -6.24636829e-01]]


In [12]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)


In [13]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

epochs = 3
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')


In [14]:
classifier_model = build_classifier_model()

In [15]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds,
                               validation_data=val_ds,
                               epochs=epochs)            

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/10


200/200 [==============================] - 738s 4s/step - loss: 0.5473 - binary_accuracy: 0.7837 - val_loss: 0.5488 - val_binary_accuracy: 0.7868
Epoch 2/10
200/200 [==============================] - 675s 3s/step - loss: 0.5074 - binary_accuracy: 0.7880 - val_loss: 0.5055 - val_binary_accuracy: 0.7868
Epoch 3/10
200/200 [==============================] - 671s 3s/step - loss: 0.4603 - binary_accuracy: 0.7989 - val_loss: 0.5246 - val_binary_accuracy: 0.7837
Epoch 4/10
200/200 [==============================] - 672s 3s/step - loss: 0.4245 - binary_accuracy: 0.8134 - val_loss: 0.5510 - val_binary_accuracy: 0.7781
Epoch 5/10
200/200 [==============================] - 671s 3s/step - loss: 0.3781 - binary_accuracy: 0.8348 - val_loss: 0.6081 - val_binary_accuracy: 0.7875
Epoch 6/10
200/200 [==============================] - 650s 3s/step - loss: 0.3348 - binary_accuracy: 0.8516 - val_loss: 0.6577 - val_binary_accuracy: 0.7912
Epoch 7/10
200/200 [==============================] - 281s 1s/step - 

In [16]:
loss, accuracy = classifier_model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

63/63 [==============================] - 23s 365ms/step - loss: 0.7817 - binary_accuracy: 0.7539
Loss: 0.7816520929336548
Accuracy: 0.7538847327232361


In [17]:
classifier_model.save_weights('model.st')

In [18]:
debug([pre['isDP'][i] for i in list(pre['isDP'])].count(True))
debug(len(pre['isDP']))

DEBUG 1713
DEBUG 7976


In [19]:
df = pd.read_csv('../data/filtered_test_data.csv')
df = df[['Statement', 'tags']]
df['isDP'] = ['dp' in i for i in df['tags'].tolist()]
df = df.drop('tags', axis = 1)
df

,Statement,isDP
0,hard version problem differ version k 12 make ...,False
1,problem differ next one presenc constraint equ...,False
2,array b call subarray form continu subsequ equ...,False
3,athenaeu finish creat latest music composit pr...,True
4,got posit integ sequenc a1 a2 number sequenc d...,True
...,...,...
1990,100 room arrang row 99 door door connect room ...,False
1991,littl artyom decid studi probabl theori found ...,True
1992,given sequenc a1 a2 queri lj rj 1 lj rj n quer...,False
1993,grid n row column three type cell empti cell d...,False


In [20]:
now = df.to_dict()
n = len(now['Statement'])
cd(root)
cd('aclImdb')
cd ('test')
for tag in ['neg', 0], ['pos', 1]:
    cd(tag[0])
    for i in range(n):
        if now['isDP'][i] ^ tag[1]: continue;
        info(f'[{i} / {n}] Parsing Rows', end = '\r')
        write_to_file(f'{i}.txt', now['Statement'][i])
    info(f'[OK] Task Parse {tag[0]} Finished.')
    cd('..')
cd('../..')

INFO  [OK] Task Parse neg Finished.
INFO  [OK] Task Parse pos Finished.
